Verschlüsselungs Beispiele in Python
==========================
Im Folgenden werden die grundlegenden Operationen in verschiedenen Python Bibliotheken ausprobiert.

In [1]:
import numpy as np
import tenseal as ts

Pyfhel Bibliothek
-----------------
https://pyfhel.readthedocs.io/en/latest/index.html

In dem ersten Schritt wird ein Pyfhel Objekt mit einem Schlüsselpaar erstellt, das in den weiteren Schritten die Verschlüsselung und Entschlüsselung ausführen wird.

In [3]:
from Pyfhel import Pyfhel

pyfhel = Pyfhel()
pyfhel.contextGen(p=65537)

pyfhel.keyGen()

Zwei Zahlen werden definiert und dann anschließend verschlüsselt.

In [4]:
integer1 = 2
integer2 = 2
ctxt1 = pyfhel.encryptInt(integer1)
ctxt2 = pyfhel.encryptInt(integer2)

Operationen wie +, - und * können auf den verschlüsselten Daten in dieser Bibliothek ausgeführt werden. Dies ist auch möglich innerhalb von Numpy Vektoren.

In [5]:
ctxtSum = ctxt1 + ctxt2        
ctxtSub = ctxt1 - ctxt2         
ctxtMul = ctxt1 * ctxt2

a = np.array([ctxt1, ctxt2])
b = np.array([ctxt2, ctxt2])
c = a + b

Im letzten Schritt werden noch die Ergebnisse entschlüsselt.

In [6]:
resSum = pyfhel.decryptInt(ctxtSum)
resSub = pyfhel.decryptInt(ctxtSub)
resMul = pyfhel.decryptInt(ctxtMul)
idx0 = pyfhel.decryptInt(c[0])
idx1 = pyfhel.decryptInt(c[1])
print(f"addition:       decrypt(ctxt1 + ctxt2) =  {resSum}")
print(f"substraction:   decrypt(ctxt1 - ctxt2) =  {resSub}")
print(f"multiplication: decrypt(ctxt1 * ctxt2) =  {resMul}")
print(f"numpy:          decrypt([ctxt1, ctxt2] + [ctxt2, ctxt2]) = {idx0,idx1}")

addition:       decrypt(ctxt1 + ctxt2) =  4
substraction:   decrypt(ctxt1 - ctxt2) =  0
multiplication: decrypt(ctxt1 * ctxt2) =  4
numpy:          decrypt([ctxt1, ctxt2] + [ctxt2, ctxt2]) = (4, 4)


Python-Paillier Bibliothek
---
https://python-paillier.readthedocs.io/en/develop/

In [7]:
from phe import paillier

public_key, private_key = paillier.generate_paillier_keypair()

In [8]:
secret_number_list = [2.0]
encrypted_number_list = [public_key.encrypt(x) for x in secret_number_list]

In [9]:
[private_key.decrypt(x) for x in encrypted_number_list]

[2.0]

TenSEAL Bibliothek
---
https://github.com/OpenMined/TenSEAL

In [10]:
bits_scale = 26

# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[31, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, 31]
)

# set the scale
context.global_scale = pow(2, bits_scale)

# galois keys are required to do ciphertext rotations
context.generate_galois_keys()

v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

skalar = [1, 1, 2, 2, 0]

# encrypted vectors
enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)

result = enc_v1 + enc_v2
result.decrypt() # ~ [4, 4, 4, 4, 4]

result = enc_v1.dot(enc_v2)
result.decrypt() # ~ [10]

result = enc_v1 * np.array(skalar) + [1, 1, 1, 1, 1]
result.decrypt()

[0.9999905657379802,
 2.0009927010841304,
 5.003905791437214,
 7.005892906271997,
 0.999921293258307]

Ungefähre Approximation des tanh mit verschlüsselten Daten

In [11]:
test = 1

print(np.tanh(test))

x = ts.ckks_vector(context, [test])
def approTanh(x):
    first = (x*x*x)
    return x - (first*(1/3))
result = approTanh(x)
print(result.decrypt())

0.7615941559557649
[0.6650418367127828]


Als nächstes ein Beispiel der Multiplikationstiefe von Leveled HE und wie oft Werte neu entschlüsselt und verschlüsselt werden müssen. Zuerst wird ein Kontext erstellt, welcher alle Multiplikationen durchführen kann. Danach wird ein kleiner Kontext verwendet. Die Datei he_memory_usage.py zeigt aber, dass der große Kontext um einiges mehr benötigt an Speicherplatz.

In [10]:
poly_mod_degree = 2 ** 14
bits_scale = 40

coeff_mod_bit_sizes=[60, bits_scale, bits_scale, bits_scale, bits_scale, 60]
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

ctx_training.global_scale = pow(2, bits_scale)
ctx_training.generate_galois_keys()

In [4]:
a = [2]

a = ts.ckks_vector(ctx_training, a)

z1 = a * a + a * a
z2 = a * a + a * a
z1 = z1 * z1
z2 = z2 * z2
z3 = z1 * a + z2 * a
z4 = z3 * z3
z4.decrypt()

[65537.56552328116]

In [2]:
poly_mod_degree = 2 ** 13
bits_scale = 40

coeff_mod_bit_sizes=[60, bits_scale, 60]
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

ctx_training.global_scale = pow(2, bits_scale)
ctx_training.generate_galois_keys()

In [3]:
a = [2]

a = ts.ckks_vector(ctx_training, a)

def recrypt(variable):
    decrypt = variable.decrypt()
    return ts.ckks_vector(ctx_training,decrypt)

z1 = a * a + a * a
z2 = a * a + a * a

z1 = recrypt(z1)
z2 = recrypt(z2)

z1 = z1 * z1
z2 = z2 * z2

z1 = recrypt(z1)
z2 = recrypt(z2)

z3 = z1 * a + z2 * a

z3 = recrypt(z3)

z4 = z3 * z3
z4.decrypt()

[65536.07922127847]